In [10]:
import numpy as np
import sqlite3
import csv
import os

In [11]:
#SQL
CREATE_DR5_SNR100_SQL = """
CREATE TABLE IF NOT EXISTS dr5_SNR100_data(
catalogue_obsid INT PRIMARY KEY NOT NULL,
catalogue_class TEXT,
catalogue_subclass TEXT, 
stellar_teff FLOAT,
stellar_logg FLOAT,
stellar_feh FLOAT,
class_label INT,
subclass_label INT);
"""

if os.path.isfile("dr5_SNR100.db"): #Checks if sqlite database already exsits.
    os.system("rm -rf dr5_SNR100.db") #If exsits, delete it.
db_sqlite = sqlite3.connect('dr5_SNR100.db')
sqlite_cursor = db_sqlite.cursor() # Creates a Cursor, that allows us to issue Create Table Stataments
sqlite_cursor.execute(CREATE_DR5_SNR100_SQL)

with open('dr5_SNR500.csv','r') as filein: 
    # csv.DictReader uses first line in file for column headings by default
    data = csv.DictReader(filein, delimiter = '|') 
    to_db = [(int(row["catalogue_obsid"]), row["catalogue_class"], row["catalogue_subclass"],\
              None if row["stellar_teff"]=='' else float(row["stellar_teff"]),\
              None if row["stellar_logg"]=='' else float(row["stellar_logg"]),\
              None if row["stellar_feh"]=='' else float(row["stellar_feh"]))\
             for row in data]
sqlite_cursor.executemany("INSERT INTO dr5_SNR100_data (catalogue_obsid,catalogue_class,catalogue_subclass,\
                            stellar_teff,stellar_logg,stellar_feh) VALUES (?,?,?,?,?,?);", to_db)
db_sqlite.commit()

In [12]:
# Select the #1 data row from our database to have a first look
sql_test="""
SELECT *
FROM dr5_SNR100_data
LIMIT 1
"""
sqlite_cursor.execute(sql_test)
data_example = sqlite_cursor.fetchone()
print("The first example of data is ", data_example)

The first example of data is  (103032, 'STAR', 'G2', 5909.24, 4.19, 0.067, None, None)


In [13]:
# select all kinds of classes
sql_classes_set_all="""
SELECT
    DISTINCT(catalogue_subclass)
FROM
    dr5_SNR100_data
"""
sqlite_cursor.execute(sql_classes_set_all)
classes_set_all = sqlite_cursor.fetchall()
print("Classes of all spectra : \n",classes_set_all)
print("Number of classes = ", len(classes_set_all))

Classes of all spectra : 
 [('G2',), ('F5',), ('F2',), ('G3',), ('F0',), ('G0',), ('A2IV',), ('A7V',), ('A1V',), ('F6',), ('A1IV',), ('A6IV',), ('A9V',), ('A8III',), ('A2V',), ('F9',), ('F3',), ('A5V',), ('B9',), ('B6',), ('A3IV',), ('F4',), ('A0III',), ('F8',), ('A7IV',), ('F7',), ('A7III',), ('G4',), ('G8',), ('G1',), ('Non',), ('G7',), ('K3',), ('K1',), ('G5',), ('G6',), ('CV',), ('A3V',), ('G9',), ('K0',), ('A6V',), ('Carbon',), ('WD',), ('A0',)]
Number of classes =  44


In [14]:
# select individual class from all data
sql_individual_class="""
SELECT
    catalogue_subclass
FROM
    dr5_SNR100_data

"""
sqlite_cursor.execute(sql_individual_class)
individual_class = sqlite_cursor.fetchall()

In [15]:
# select all obsids as updata indicators.
sqlite_cursor.execute("SELECT catalogue_obsid FROM dr5_SNR100_data")
obsids = sqlite_cursor.fetchall()
#label spectra with their class and subclass
update_tuples_list=[] # consists of (id,class_label,subclass_label)
for i in range(len(individual_class)):
    if individual_class[i] == ("Non",):
        classes_label = -1
    elif individual_class[i] == ("WD",):
        classes_label = 0
    elif individual_class[i] == ("Carbon",):
        classes_label = 1
    elif individual_class[i] == ("CV",):
        classes_label = 2
    elif individual_class[i][0][0] == "B":
        classes_label = 3 
    elif individual_class[i][0][0] == "A":
        classes_label = 4
    elif individual_class[i][0][0] == "F":
        classes_label = 5
    elif individual_class[i][0][0] == "G":
        classes_label = 6
    elif individual_class[i][0][0] == "K":
        classes_label = 7
    elif individual_class[i][0][0] == "M":
        classes_label = 8
    else:
        classes_label = 9
    
    subclasses_label = classes_set_all.index(individual_class[i])
    update_tuples_list.append((classes_label,subclasses_label,obsids[i][0]))
print("the tuple(class_label, subclass_label, obsid) for updating is like :", update_tuples_list[0])

the tuple(class_label, subclass_label, obsid) for updating is like : (6, 0, 103032)


In [16]:
# Update labels to databese 
sqlite_cursor.executemany("UPDATE dr5_SNR100_data SET class_label=? , \
subclass_label = ? WHERE catalogue_obsid=?;", update_tuples_list)
db_sqlite.commit()

In [17]:
# Have a look of our updated database.
sqlite_cursor.execute(sql_test)
updated_example = sqlite_cursor.fetchone()
print("updated data example: ",updated_example)

updated data example:  (103032, 'STAR', 'G2', 5909.24, 4.19, 0.067, 6, 0)


In [27]:
#Select data for class-classification
sql_obsid_classlabel="""
SELECT
    catalogue_obsid, class_label
FROM
    dr5_SNR100_data
WHERE
    class_label != -1
"""
sqlite_cursor.execute(sql_obsid_classlabel)
obsid_classlabel = sqlite_cursor.fetchall()
with open("dr5_SNR100_obsid_classlabel.csv", 'w', newline='') as fileout_class:
    writer = csv.writer(fileout_class)
    for row in obsid_classlabel:
        writer.writerow(row)

In [31]:
#Select data for class-classification
sql_obsid_subclasslabel="""
SELECT
    catalogue_obsid, subclass_label
FROM
    dr5_SNR100_data
WHERE
    class_label != -1
"""
sqlite_cursor.execute(sql_obsid_subclasslabel)
obsid_subclasslabel = sqlite_cursor.fetchall()
with open("dr5_SNR100_obsid_subclasslabel.csv", 'w', newline='') as fileout_subclass:
    writer = csv.writer(fileout_subclass)
    for row in obsid_subclasslabel:
        writer.writerow(row)

In [35]:
# Select data for linear regression of T_eff and log(g)
sql_obsid_teff_logg = """
SELECT 
    catalogue_obsid, stellar_teff, stellar_logg
FROM 
    dr5_SNR100_data
WHERE
    stellar_teff IS NOT NULL AND
    stellar_logg IS NOT NULL
"""
sqlite_cursor.execute(sql_obsid_teff_logg)
obsid_teff_logg = sqlite_cursor.fetchall()
with open("dr5_SNR100_obsid_teff_logg.csv", 'w', newline='') as fileout_teff_logg:
    writer = csv.writer(fileout_teff_logg)
    for row in obsid_teff_logg:
        writer.writerow(row)

In [39]:
len(obsid_teff_logg)

30817

In [37]:
a = np.array([1,2,3])
b = np.array([1,2])
c = np.in1d(a,b)
print(c)
print(a[c])

[ True  True False]
[1 2]
